<img src='../logos/EUMETSAT_Logo_WhiteonBlue.png' alt='logo EUMETSAT' align='center' width='30%'></img>

# 
# Retrieve images from EUMETView 

**[EUMETView](https://view.eumetsat.int/productviewer?v=default)** is an Online Map Service that provides visualisations of EUMETSAT product through a customisable web user interface and an enhanced set of Open Geospatial Consortium (OGC) standard API. EUMETView makes it possible to create and save maps using the user interface, or integrates with user's personal service, via the API.



<div class="alert alert-block alert-info">
<b>NOTE:</b> To successfully run this notebook please follow the instructions, namely those within the blue cells like this one, so that you know what you need to change. For the other cells you should only hit the play button, to run them.</div>

<hr>

#### Install and load required libraries

In [ ]:
!pip install owslib

In [ ]:
import owslib
from datetime import datetime, timedelta
import glob#
import zipfile

<div class="alert alert-block alert-info">
<b>Define the outupt folder (where the downloaded images will be saved).</div>

In [ ]:
Service='mtg_fd'


<div class="alert alert-block alert-info">
<b>Define one Product by uncommenting the correct line in the cell below.</div>

In [ ]:
#product_name='rgb_truecolour'
product_name='rgb_cloudphase'
#product_name='rgb_dust'
#product_name='rgb_geocolour'



In [ ]:
output_folder='EUMETVIEW_IMAGERY/'+product_name+'/'

!mkdir $output_folder



In [ ]:
target_layer = Service+':'+product_name+',backgrounds:ne_10m_coastline'


#### Select a Geographical Area


<div class="alert alert-block alert-info">
<b>In the next cell you can change select your Area Of Interest by entering iths geographical limits. Please also give a name to your region by editing the variable RegionName.
<br>
    
>- **`'RegionName='`** <br>
>- **`'lon_min='`** <br>
>- **`'lon_max='`** <br>
>- **`'lat_min='`** <br>
>- **`'lat_max='`** <br>


In [ ]:

#RegionName='Iberia'
#lon_min=-8.
#lon_max=3
#lat_min=35.
#lat_max=45.

RegionName='AFR'
lon_min=19.
lat_min=-12
lon_max=63.
lat_max=15.5



In [ ]:
bbox=(lon_min,lat_min,lon_max,lat_max)
xsize=int((lon_max-lon_min)+1)*50
ysize=int((lat_max-lat_min)+1)*50
size=(xsize,ysize)


In [ ]:
from owslib.wms import WebMapService 
service_url = 'https://view.eumetsat.int/geoserver/ows?'
wms = WebMapService(service_url)


In [ ]:
def get_wms_image (dt,wms_layer,bbox,size):
    
    timewindow=(dt+timedelta(hours=-0.25)).strftime("%Y-%m-%dT%H:%M:%SZ")+"/"+(dt.strftime("%Y-%m-%dT%H:%M:%SZ")) 
    #print(timewindow)
        # setup a 15 min window - will return teh last time
    payload = {
        'layers' : [wms_layer],
        'styles' : '',
        'format' : 'image/jpeg', # format 
        'crs'    : 'EPSG:4326', # projection
        'bbox'   : bbox, # BBOX and Size needs to match
        'size'   : size,
        'time' : timewindow
    }

    wms = WebMapService(service_url)
    img_response = wms.getmap(**payload)
    img=img_response.read()
    
    return img



<div class="alert alert-block alert-info">
<b>In the next cell you can select the date range.
<br>
    
>- **`'YEAR='`** <br>
>- **`'MONTH='`** <br>
>- **`'DAY_INI='`** <br>
>- **`'DAY_FIN='`** <br>
>- **`'H_INI='`** <br>
>- **`'H_FIN='`** <br>
>- **`'MIN_INI='`** <br>
>- **`'MIN_FIN='`** <br>
>- **`'DELTA_MIN='`** <br></div>

   

In [ ]:
YEAR=2025
MONTH=5
DAY_INI=16
DAY_FIN=17
H_INI=12
H_FIN=12
M_INI=0
M_FIN=30
DELTA_MIN=10


In [ ]:
def datespan(startDate, endDate, delta=timedelta(days=1)):
    currentDate = startDate
    while currentDate < endDate:
        yield currentDate
        currentDate += delta

In [ ]:
print(output_folder)
for timestamp in datespan(datetime(YEAR, MONTH, DAY_INI, H_INI, M_INI), 
                          datetime(YEAR, MONTH, DAY_FIN, H_FIN, M_FIN), 
                          delta=timedelta(minutes=DELTA_MIN)):
    print(timestamp)
 
    filename=timestamp.strftime("%Y-%m-%dT%H.%M.00Z")+".jpg"
    
    img=get_wms_image (timestamp,target_layer,bbox,size)
    with open(output_folder+filename, 'wb') as f:
        f.write(img)
    f.close()

<hr>


    
<p style="text-align:right;"> <a href='https://training.eumetsat.int'>EUMETSAT Training</a> | <a href='mailto:training@eumetsat.int'>Contact the training team</a></p>